#### Pandas数据结构—合并、连接、修补、去重、替换
##### 一、合并（一） merge
##### 二、合并（二）join
##### 三、连接 concat
##### 四、修补 combine_first、updata
##### 五、去重 duplicated
##### 六、替换 replace

In [1]:
import pandas as pd
import numpy as np

###### 一、合并（一）-merge
###### merge有点类似vlookup，但是比vlookup更灵活。
###### merge的参数有很多：
###### left, right, how='inner', on=None, left_on=None, right_on=None,left_index=False, right_index=False, 
###### sort=True, suffixes=('_x', '_y'), copy=True, indicator=False 

In [2]:
df1 = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})
df2 = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})
df3 = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                    'key2': ['K0', 'K1', 'K0', 'K1'],
                    'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3']})
df4 = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                    'key2': ['K0', 'K0', 'K0', 'K0'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']})
# 可以看到df1和df2都有key这个列，并且数据都相同。
print(df1)
print('--------------------------------')
print(df2)
print('--------------------------------')
# df3和df4都有key1列和key2列，但是有一部分数据不同。
print(df3)
print('--------------------------------')
print(df4)
print('--------------------------------')

  key   A   B
0  K0  A0  B0
1  K1  A1  B1
2  K2  A2  B2
3  K3  A3  B3
--------------------------------
  key   C   D
0  K0  C0  D0
1  K1  C1  D1
2  K2  C2  D2
3  K3  C3  D3
--------------------------------
  key1 key2   A   B
0   K0   K0  A0  B0
1   K0   K1  A1  B1
2   K1   K0  A2  B2
3   K2   K1  A3  B3
--------------------------------
  key1 key2   C   D
0   K0   K0  C0  D0
1   K1   K0  C1  D1
2   K1   K0  C2  D2
3   K2   K0  C3  D3
--------------------------------


###### 1、参数 left 、right、on
###### left：准备合并的第一组数据
###### right：准备合并的第二组数据
###### on：参考键

In [3]:
# 通过设置参数on的值来确定合并的参考键。也就说以什么作为参考。on = 'key'就是以key作为参考。
print('将df1和df2进行合并，合并的原则是根据参考键key： \n',pd.merge(df1, df2, on = 'key'))
print('--------------------------------')

# df3和df4的key1和key2的第一行数据相同，所以进行合并。
# df3的key1和key2的第二行数据，在df4的key1和key2列中没有相同的数据，所以不进行合并。
# 注意：df3的key1和key2的第三行数据，和df4的key1和key2列中有两行相同的数据，所以会进行两次合并。
# # df3的key1和key2的第四行数据，在df4的key1和key2列中没有相同的数据，所以不进行合并。
print(pd.merge(df3, df4, on = ['key1','key2']))

将df1和df2进行合并，合并的原则是根据参考键key： 
   key   A   B   C   D
0  K0  A0  B0  C0  D0
1  K1  A1  B1  C1  D1
2  K2  A2  B2  C2  D2
3  K3  A3  B3  C3  D3
--------------------------------
  key1 key2   A   B   C   D
0   K0   K0  A0  B0  C0  D0
1   K1   K0  A2  B2  C1  D1
2   K1   K0  A2  B2  C2  D2


###### 2、参数how
###### how：合并方式

In [4]:
# how = 'inner'：表示取的是交集
print(pd.merge(df3, df4,on=['key1','key2'], how = 'inner'))  
print('--------------------------------')

# how = 'outer'：表示取的是并集，如果数据缺失就用NaN填充
print(pd.merge(df3, df4, on=['key1','key2'], how = 'outer'))  
print('--------------------------------')

# how = 'left'：表示将第一组准备合并的数据为参考进行合并（本例中是df3），如果数据缺失就用NaN填充
print(pd.merge(df3, df4, on=['key1','key2'], how = 'left'))  
print('--------------------------------')

# how = 'right'：表示将第二组准备合并的数据为参考进行合并（本例中是df4），如果数据缺失就用NaN填充
print(pd.merge(df3, df4, on=['key1','key2'], how = 'right'))  

  key1 key2   A   B   C   D
0   K0   K0  A0  B0  C0  D0
1   K1   K0  A2  B2  C1  D1
2   K1   K0  A2  B2  C2  D2
--------------------------------
  key1 key2    A    B    C    D
0   K0   K0   A0   B0   C0   D0
1   K0   K1   A1   B1  NaN  NaN
2   K1   K0   A2   B2   C1   D1
3   K1   K0   A2   B2   C2   D2
4   K2   K1   A3   B3  NaN  NaN
5   K2   K0  NaN  NaN   C3   D3
--------------------------------
  key1 key2   A   B    C    D
0   K0   K0  A0  B0   C0   D0
1   K0   K1  A1  B1  NaN  NaN
2   K1   K0  A2  B2   C1   D1
3   K1   K0  A2  B2   C2   D2
4   K2   K1  A3  B3  NaN  NaN
--------------------------------
  key1 key2    A    B   C   D
0   K0   K0   A0   B0  C0  D0
1   K1   K0   A2   B2  C1  D1
2   K1   K0   A2   B2  C2  D2
3   K2   K0  NaN  NaN  C3  D3


###### 3、参数 left_on, right_on, left_index, right_index 
###### 当键不同时，可以通过left_on, right_on分别设置左键与右键。当键相同时，直接用on
###### 当两组数据的列名不同时可以使用left_index, right_index来进行合并操作。
###### left_on, right_on, left_index, right_index可以进行相互的组合：
###### left_on + right_on, left_on + right_index, left_index + right_on, left_index + right_index

In [5]:
df1 = pd.DataFrame({'xkey':list('abcdefs'),
                   'data1':range(7)})
df2 = pd.DataFrame({'ykey':list('abd'),
                   'date2':range(3)})
print(df1)
print('--------------------------------')
print(df2)
print('--------------------------------')

# 通过left_on='xkey', right_on='ykey'分别设置了合并所需的左键和右键。
# 在合并的时候，依然会比较这两个键里的具体数据是否有相同的，如果有就进行合并。
print(pd.merge(df1, df2, left_on='xkey', right_on='ykey'))
print('--------------------------------')


df3 = pd.DataFrame({'key':list('abcdfeg'),
                   'data1':range(7)})
df4 = pd.DataFrame({'data2':range(10,15)},
                  index = list('abcde'))
print(df3)
print('--------------------------------')
print(df4)
print('--------------------------------')

# df3有key和data1两列，df4除了index就只有data2一列，但是index和df3的key有部分相同。
# 将df3设置成以‘key’为键，而df4设置成以index为键。
# left_index：为True时，第一组数据是以index为键，默认False
# right_index：为True时，第二组数据是以index为键，默认False
print(pd.merge(df3, df4, left_on='key', right_index=True))

  xkey  data1
0    a      0
1    b      1
2    c      2
3    d      3
4    e      4
5    f      5
6    s      6
--------------------------------
  ykey  date2
0    a      0
1    b      1
2    d      2
--------------------------------
  xkey  data1 ykey  date2
0    a      0    a      0
1    b      1    b      1
2    d      3    d      2
--------------------------------
  key  data1
0   a      0
1   b      1
2   c      2
3   d      3
4   f      4
5   e      5
6   g      6
--------------------------------
   data2
a     10
b     11
c     12
d     13
e     14
--------------------------------
  key  data1  data2
0   a      0     10
1   b      1     11
2   c      2     12
3   d      3     13
5   e      5     14


###### 4、参数 sort
###### 按照字典的顺序对结果进行排序。默认为False，设置为False会大幅提高性能

In [6]:
df1 = pd.DataFrame({'key':list('abcdefg'),
                   'data1':[1,3,2,4,5,9,7]})
df2 = pd.DataFrame({'key':list('abd'),
                   'data2':[1,2,3]})
print(df1)
print('--------------------------------')
print(df2)
print('--------------------------------')

# 将df1和df2以key为键，进行合并，并且取并集。
pm1 = pd.merge(df1,df2, on = 'key', how = 'outer')

# 将df1和df2以key为键，进行合并，并且取交集，而且还要进行排序。
pm2 = pd.merge(df1,df2, on = 'key', sort=True, how = 'outer')
print(pm1)
print('--------------------------------')
print(pm2)
print('--------------------------------')

  key  data1
0   a      1
1   b      3
2   c      2
3   d      4
4   e      5
5   f      9
6   g      7
--------------------------------
  key  data2
0   a      1
1   b      2
2   d      3
--------------------------------
  key  data1  data2
0   a      1    1.0
1   b      3    2.0
2   c      2    NaN
3   d      4    3.0
4   e      5    NaN
5   f      9    NaN
6   g      7    NaN
--------------------------------
  key  data1  data2
0   a      1    1.0
1   b      3    2.0
2   c      2    NaN
3   d      4    3.0
4   e      5    NaN
5   f      9    NaN
6   g      7    NaN
--------------------------------


###### 二、合并（二）-join

In [15]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                     'B': ['B0', 'B1', 'B2']},
                    index=['K0', 'K1', 'K2'])
df2 = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                      'D': ['D0', 'D2', 'D3']},
                     index=['K0', 'K2', 'K3'])
print(df1)
print('--------------------------------')
print(df2)
print('--------------------------------')

# 以index为键将df1和df2进行合并。其中df1是为主，也就是说以df1为准备，然后根据df2进行合并。具体说明如下：
# df1的index有'K0', 'K1', 'K2'
# df2的index有'K0', 'K2', 'K3'
# 因为df1和df2中都'K0'和'K2',所以这两行就直接合并了。而K1因为只在df1中有，所以合并后缺失值用NaN来填充。
print(df1.join(df2))
print('--------------------------------')

# 下面这句等价于：pd.merge(left, right, left_index=True, right_index=True, how='outer')
# 注意：因为how='outer'所以取的是交集。也就是说有对应的数据就合并，没有的就用NaN来填充缺失值。
print(df1.join(df2, how='outer'))  
print('--------------------------------')

     A   B
K0  A0  B0
K1  A1  B1
K2  A2  B2
--------------------------------
     C   D
K0  C0  D0
K2  C2  D2
K3  C3  D3
--------------------------------
     A   B    C    D
K0  A0  B0   C0   D0
K1  A1  B1  NaN  NaN
K2  A2  B2   C2   D2
--------------------------------
      A    B    C    D
K0   A0   B0   C0   D0
K1   A1   B1  NaN  NaN
K2   A2   B2   C2   D2
K3  NaN  NaN   C3   D3
--------------------------------
     A   B    D
K0  A0  B0   D0
K1  A1  B1  NaN
K2  A2  B2   D2
--------------------------------


###### 1、参数suffixes
###### 为合并后相同的键进行区分.默认的是('_x', '_y')。

In [16]:
df1 = pd.DataFrame({'key':list('abcdefg'),
                   'data1':[1,3,2,4,5,9,7]})
df2 = pd.DataFrame({'key':list('abd'),
                   'data2':[1,2,3]})
print(df1)
print('--------------------------------')
print(df2)
print('--------------------------------')

# 因为df1和df2中都有key，所以通过suffixes来区分。如果不写suffixes那么就用默认值来区分。
print(pd.merge(df1, df2, left_index=True, right_index=True)) 
print('--------------------------------')
# df1的key用key_666，df2的key用key_777
print(pd.merge(df1, df2, left_index=True, right_index=True, suffixes=('_666', '_777'))) 
print('--------------------------------')

  key  data1
0   a      1
1   b      3
2   c      2
3   d      4
4   e      5
5   f      9
6   g      7
--------------------------------
  key  data2
0   a      1
1   b      2
2   d      3
--------------------------------
  key_x  data1 key_y  data2
0     a      1     a      1
1     b      3     b      2
2     c      2     d      3
--------------------------------
  key_666  data1 key_777  data2
0       a      1       a      1
1       b      3       b      2
2       c      2       d      3
--------------------------------


###### 2、参数on

In [3]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'key': ['K0', 'K1', 'K0', 'K1']})
df2 = pd.DataFrame({'C': ['C666', 'C777'],
                      'D': ['D666', 'D777']},
                     index=['K0', 'K1'])
print(df1)
print('--------------------------------')
print(df2)
print('--------------------------------')

# join默认是用index作为键，但是可以通过on参数来调整。
# df1用key作为键，df2依然使用index。
# 注意：因为指定了df1的key作为键，而df1的key是'K0', 'K1', 'K0', 'K1'。所以df2根据key的值做了两次合并。
# 下面这句等价于：pd.merge(left, right, left_on='key', right_index=True, how='left', sort=False);
print(df1.join(df2, on = 'key'))

    A   B key
0  A0  B0  K0
1  A1  B1  K1
2  A2  B2  K0
3  A3  B3  K1
--------------------------------
       C     D
K0  C666  D666
K1  C777  D777
--------------------------------
    A   B key     C     D
0  A0  B0  K0  C666  D666
1  A1  B1  K1  C777  D777
2  A2  B2  K0  C666  D666
3  A3  B3  K1  C777  D777


###### 三、连接-concat
###### concat参数：
###### objs, axis=0, join='outer', join_axes=None, ignore_index=False,keys=None, levels=None, names=None, verify_integrity=False,copy=True

In [18]:
ps1 = pd.Series([1,2,3])
ps2 = pd.Series([2,3,4,5])
print(ps1)
print('--------------------------------')
print(ps2)
print('--------------------------------')
# 通过concat将ps1和ps2进行连接。
print(pd.concat([ps1,ps2]).)

0    1
1    2
2    3
dtype: int64
--------------------------------
0    2
1    3
2    4
3    5
dtype: int64
--------------------------------
0    1
0    2
1    2
1    3
2    3
2    4
3    5
dtype: int64


###### 1、参数axis

In [19]:
# axis默认是0，也就行+行，如果=1那就是列+列
ps3 = pd.Series([1,2,3],index = ['a','c','g'])
ps4 = pd.Series([2,3,4,5],index = ['b','e','d','f'])
print(pd.concat([ps3,ps4]))
print('--------------------------------')

# 通过sort_index里控制是否要对index进行排序。注意与上面的对比。
print(pd.concat([ps3,ps4]).sort_index())
print('--------------------------------')

# 通过设置axis=1来设置按列连接，对于空值用NaN来填充。
print(pd.concat([ps3,ps4], axis=1))
print('--------------------------------')

a    1
c    2
g    3
b    2
e    3
d    4
f    5
dtype: int64
--------------------------------
a    1
b    2
c    2
d    4
e    3
f    5
g    3
dtype: int64
--------------------------------
     0    1
a  1.0  NaN
b  NaN  2.0
c  2.0  NaN
d  NaN  4.0
e  NaN  3.0
f  NaN  5.0
g  3.0  NaN
--------------------------------


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [23]:
# concat对DataFrame的用法一样
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'key': ['K0', 'K1', 'K0', 'K1']})
df2 = pd.DataFrame({'C': ['C666', 'C777'],
                      'D': ['D666', 'D777']},
                     index=['K0', 'K1'])
print(pd.concat([df1,df2]))

      A    B     C     D  key
0    A0   B0   NaN   NaN   K0
1    A1   B1   NaN   NaN   K1
2    A2   B2   NaN   NaN   K0
3    A3   B3   NaN   NaN   K1
K0  NaN  NaN  C666  D666  NaN
K1  NaN  NaN  C777  D777  NaN


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


###### 2、参数join，join_axes

In [56]:
# join
ps5 = pd.Series([1,2,3],index = ['a','b','c'])
ps6 = pd.Series([4,5,6],index = ['c','b','d'])

# 参数join默认是outer,直接进行合并。如果设置成inner则会取交集。
print(pd.concat([ps5,ps6], axis= 1, join='outer'))
print('--------------------------------')
print(pd.concat([ps5,ps6], axis= 1, join='inner'))
print('--------------------------------')

# 参数join_axes是指定联合的index
print(pd.concat([s5,s6], axis = 1, join_axes=[['a','b','d']]))

     0    1
a  1.0  NaN
b  2.0  5.0
c  3.0  4.0
d  NaN  6.0
--------------------------------
   0  1
b  2  5
c  3  4
--------------------------------
     0    1
a  1.0  NaN
b  2.0  NaN
d  NaN  4.0


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


###### 3、参数keys

In [24]:
ps5 = pd.Series([1,2,3],index = ['a','b','c'])
ps6 = pd.Series([4,5,6],index = ['c','b','d'])
print(ps5)
print('--------------------------------')
print(ps6)
print('--------------------------------')

# 通过参数keys来指定最外层构建层的索引
ps7 = pd.concat([ps5, ps6], keys = ['A', 'B'])
print('--------------------------------')
print(ps7)
print('--------------------------------')

# 通过下标方式来访问索引。
# 下面得到了两个index，一个是'A'一个'a'
print(ps7.index[0])
print('--------------------------------')
# 得到最外层的索引，'A'。
print(ps7.index[0][0])
print('--------------------------------')
# 如果使用了axis=1这个参数，那么keys的值就是columns。
print(pd.concat([ps5, ps6], axis = 1, keys = ['A', 'B']))
print('--------------------------------')
# 查看一下columns。
print(pd.concat([ps5, ps6], axis = 1, keys = ['A', 'B']).columns)

a    1
b    2
c    3
dtype: int64
--------------------------------
c    4
b    5
d    6
dtype: int64
--------------------------------
--------------------------------
A  a    1
   b    2
   c    3
B  c    4
   b    5
   d    6
dtype: int64
--------------------------------
('A', 'a')
--------------------------------
A
--------------------------------
     A    B
a  1.0  NaN
b  2.0  5.0
c  3.0  4.0
d  NaN  6.0
--------------------------------
Index(['A', 'B'], dtype='object')


###### 四、修补 combine_first、updata

In [2]:
df1 = pd.DataFrame([[np.nan, 3, 5], [4, np.nan, np.nan],[np.nan, 6, np.nan]])
df2 = pd.DataFrame([[7, np.nan, 8], [9, 10, 666]],index=[1, 2])
print(df1)
print('--------------------------------')
print(df2)
print('--------------------------------')

# 通过combine_first来讲df2填充到df1里空值的位置上，并且是根据index来做的。
print(df1.combine_first(df2))
print('--------------------------------')

# 如果df3的index比df1的多，那么就直接更新到df1上。
df3 = pd.DataFrame([[7, np.nan, 8], [9, 10, 666]],index=['a',1])
print(df1.combine_first(df3))
print('--------------------------------')

# update，根据index的位置，将相同位置的数值用df2直接覆盖df1。
df1.update(df2)
print(df1)
print('--------------------------------')

     0    1    2
0  NaN  3.0  5.0
1  4.0  NaN  NaN
2  NaN  6.0  NaN
--------------------------------
   0     1    2
1  7   NaN    8
2  9  10.0  666
--------------------------------
     0    1      2
0  NaN  3.0    5.0
1  4.0  NaN    8.0
2  9.0  6.0  666.0
--------------------------------
     0     1      2
0  NaN   3.0    5.0
1  4.0  10.0  666.0
2  NaN   6.0    NaN
a  7.0   NaN    8.0
--------------------------------
     0     1      2
0  NaN   3.0    5.0
1  7.0   NaN    8.0
2  9.0  10.0  666.0
--------------------------------


D:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:3140: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  return this.join(other, how=how, return_indexers=return_indexers)


###### 五、去重 duplicated

In [18]:
ps = pd.Series([1,1,1,2,2,2,1,2,3,3,3,4,4,4,5,5])
print(ps)
print('--------------------------------')
# duplicated来判断数据中是否有重复的值，得到是一个布尔型的数据。
# 注意：在判断的过程中，对于重复的数据中，出现的第一个是判断为不重复的也就是(False)
print(ps.duplicated())
print('--------------------------------')

# 利用判读出的是否重复的False来得到不重复的值。
print(ps[ps.duplicated() == False])
print('--------------------------------')

# 通过drop_duplicates来直接生成去重后的数据。
ps_re = ps.drop_duplicates()
print('通过drop_duplicates直接生成去重后的数据：\n',ps_re)
print('--------------------------------')

# drop_duplicates有一个inplace参数，表示是否替换原值，默认False。也就不替换原值
ps1 = pd.Series([1,1,2,2,2,3,3,3])
print(ps1)
print('--------------------------------')

ps1_re = ps1.drop_duplicates()
print('通过drop_duplicates直接生成去重后的数据：\n',ps1_re)
print('--------------------------------')
print('inplace采用False，不替换原值，所以ps1和原来的ps1是一样的：\n',ps1)
print('--------------------------------')

ps2_re = ps1.drop_duplicates(inplace = True)
print('因为inplace=True，所以不生成新的数据，得到的就是None：\n',ps2_re)
print('--------------------------------')
print('inplace采用True，替换原值，所以ps1被修改了！！：\n',ps1)
print('--------------------------------')


df = pd.DataFrame({'key1':['a','a',3,4,5],
                  'key2':['a','a','b','d','b']})
print(df)
print('--------------------------------')

# duplicated如果用在DataFrame上的话，是比较两列是否有重复。
# 例子中key1列的第一和第二个元素与key2列的第一和第二个元素是重复的，所以结果会出现True。
print(df.duplicated())
print('--------------------------------')

# 如果用只对key2列进行判断，那么就和之前的Series一样了。
print(df['key2'].duplicated())
print('--------------------------------')

0     1
1     1
2     1
3     2
4     2
5     2
6     1
7     2
8     3
9     3
10    3
11    4
12    4
13    4
14    5
15    5
dtype: int64
--------------------------------
0     False
1      True
2      True
3     False
4      True
5      True
6      True
7      True
8     False
9      True
10     True
11    False
12     True
13     True
14    False
15     True
dtype: bool
--------------------------------
0     1
3     2
8     3
11    4
14    5
dtype: int64
--------------------------------
通过drop_duplicates直接生成去重后的数据：
 0     1
3     2
8     3
11    4
14    5
dtype: int64
--------------------------------
0    1
1    1
2    2
3    2
4    2
5    3
6    3
7    3
dtype: int64
--------------------------------
通过drop_duplicates直接生成去重后的数据：
 0    1
2    2
5    3
dtype: int64
--------------------------------
inplace采用False，不替换原值，所以ps1和原来的ps1是一样的：
 0    1
1    1
2    2
3    2
4    2
5    3
6    3
7    3
dtype: int64
--------------------------------
因为inplace=True，所以不生成新的数据，得到的就是None：
 None
----

###### 六、替换 replace

In [25]:
s = pd.Series(list('aabbccddee'))

# 通过replace将list中的a替换成空值。
print(s.replace('a', np.nan))
print('--------------------------------')

# 将list中的b和c替换成haha
print(s.replace(['b','c'],'haha'))

# 可以通过字典来同时替换多个值，也推荐这样做。
print('--------------------------------')
print(s.replace({'a':'Python!', 'd':666}))

0    NaN
1    NaN
2      b
3      b
4      c
5      c
6      d
7      d
8      e
9      e
dtype: object
--------------------------------
0       a
1       a
2    haha
3    haha
4    haha
5    haha
6       d
7       d
8       e
9       e
dtype: object
--------------------------------
0    Python!
1    Python!
2          b
3          b
4          c
5          c
6        666
7        666
8          e
9          e
dtype: object
